-----
# Initial Setup

In [27]:
import findspark
findspark.init()
findspark.find()

'C:\\Program Files\\spark-3.5.1-bin-hadoop3'

In [28]:
import pandas as pd
import numpy as np 
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from functools import reduce

In [3]:
import pyspark
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

from scipy import stats

# from pyspark.sql.types import *
from pyspark.sql.types import IntegerType, BooleanType, DateType, FloatType, StringType, ArrayType

from pyspark.sql import types
from pyspark.sql import functions as F
from pyspark.sql.functions import mean, stddev, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import year, month, hour
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode, round
from pyspark.sql.functions import coalesce, first, lit
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import datediff
from pyspark.sql.functions import when
from pyspark.sql.functions import expr

from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel
from pyspark.ml.linalg import Vectors

In [ ]:
import matplotlib.pyplot as plt

In [4]:
spark = SparkSession.builder\
            .appName("trip_records")\
            .getOrCreate()

------
# Data Import

### Improt data of NYC trip records of 2023

In [6]:
jan = spark.read \
        .parquet(r"C:\Users\USER\Desktop\Aptech_FinalProject_Sem2\Data\yellowtripdata_2023\yellow_tripdata_2023-01.parquet")
feb = spark.read \
        .parquet(r"C:\Users\USER\Desktop\Aptech_FinalProject_Sem2\Data\yellowtripdata_2023\yellow_tripdata_2023-02.parquet")
mar = spark.read \
        .parquet(r"C:\Users\USER\Desktop\Aptech_FinalProject_Sem2\Data\yellowtripdata_2023\yellow_tripdata_2023-03.parquet")
apr = spark.read \
        .parquet(r"C:\Users\USER\Desktop\Aptech_FinalProject_Sem2\Data\yellowtripdata_2023\yellow_tripdata_2023-04.parquet")
may = spark.read \
        .parquet(r"C:\Users\USER\Desktop\Aptech_FinalProject_Sem2\Data\yellowtripdata_2023\yellow_tripdata_2023-05.parquet")
jun = spark.read \
        .parquet(r"C:\Users\USER\Desktop\Aptech_FinalProject_Sem2\Data\yellowtripdata_2023\yellow_tripdata_2023-06.parquet")
jul = spark.read \
        .parquet(r"C:\Users\USER\Desktop\Aptech_FinalProject_Sem2\Data\yellowtripdata_2023\yellow_tripdata_2023-07.parquet")
aug = spark.read \
        .parquet(r"C:\Users\USER\Desktop\Aptech_FinalProject_Sem2\Data\yellowtripdata_2023\yellow_tripdata_2023-08.parquet")
sep = spark.read \
        .parquet(r"C:\Users\USER\Desktop\Aptech_FinalProject_Sem2\Data\yellowtripdata_2023\yellow_tripdata_2023-09.parquet")
oct = spark.read \
        .parquet(r"C:\Users\USER\Desktop\Aptech_FinalProject_Sem2\Data\yellowtripdata_2023\yellow_tripdata_2023-10.parquet")
nov = spark.read \
        .parquet(r"C:\Users\USER\Desktop\Aptech_FinalProject_Sem2\Data\yellowtripdata_2023\yellow_tripdata_2023-11.parquet")
dec = spark.read \
        .parquet(r"C:\Users\USER\Desktop\Aptech_FinalProject_Sem2\Data\yellowtripdata_2023\yellow_tripdata_2023-12.parquet")

In [7]:
whole_year = [jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec]

df = reduce(lambda df, df1: df.union(df1), whole_year)

In [11]:
df.count()

38310226

In [12]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [13]:
df.show(100)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2023-01-01 00:32:10|  2023-01-01 00:40:36|            1.0|         0.97|       1.0|                 N|         161|         141|           2|        9.3|  1.0|    0.5|       0.

### Import taxi zone data of NYC

In this dataset, the name of each borough and zone is corresponding to the ID of the main dataset, NYC trip records

In [8]:
zones = spark.read.option("header", True) \
             .csv(r"C:\Users\USER\Desktop\Aptech_FinalProject_Sem2\Data\taxi_zones.csv")

In [13]:
zones.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [14]:
zones.show(20)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

-----
# 1. Data Cleaning

### 1.1. Remove Unwanted Data

In [9]:
df = df.filter(
    (col('trip_distance') >0) &
    (col('trip_distance') <100) &
    (col('fare_amount') >0) &
    (col('extra') >= 0) &
    (col('mta_tax') >0) &
    (col('tip_amount') >= 0) &
    (col('tolls_amount') >= 0) &
    (col('total_amount') > 0) &
    (col('congestion_surcharge') >= 0) &
    (col('passenger_count') < 8) &
    (col('tpep_pickup_datetime') >= '2023-01-01') &
    (col('tpep_pickup_datetime') < '2024-01-01') &
    (col('tpep_dropoff_datetime') >= '2023-01-01') &
    (col('tpep_dropoff_datetime') < '2024-01-01'))

In [10]:
df.cache()

DataFrame[VendorID: bigint, tpep_pickup_datetime: timestamp_ntz, tpep_dropoff_datetime: timestamp_ntz, passenger_count: double, trip_distance: double, RatecodeID: double, store_and_fwd_flag: string, PULocationID: bigint, DOLocationID: bigint, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, airport_fee: double]

In [13]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [17]:
df.count()

35944809

##### Remove null values

In [37]:
# Find null values
from pyspark.sql.functions import sum as spark_sum
# df.select([col(column).isNull().alias(column) for column in df.columns]).sum().show()
df.select([spark_sum(col(column).isNull().cast("int")).alias(column) for column in df.columns]).show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+----------+-----------+------------+-------------+--------------+-----------+---------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|pickup_hour|dropoff_hour|dow_pickup|dow_dropoff|month_pickup|month_dropoff|pickup_borough|pickup_zone|dropoff_borough|dropoff_zone|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+----

=> No have null

### 1.3. The Additon Days-of-week (DOW) and Month Column

In [10]:
df = df.withColumn("pickup_hour", F.date_format("tpep_pickup_datetime", "HH")) \
        .withColumn("dropoff_hour", F.date_format("tpep_dropoff_datetime", "HH")) \
        .withColumn("dow_pickup", F.date_format("tpep_pickup_datetime", "EEEE")) \
        .withColumn("dow_dropoff", F.date_format("tpep_pickup_datetime", "EEEE")) \
        .withColumn("month_pickup", F.date_format("tpep_pickup_datetime", "MMMM")) \
        .withColumn("month_dropoff", F.date_format("tpep_dropoff_datetime", "MMMM"))

In [26]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- pickup_hour: string (nullable = true)
 |-- dropoff_hour: string (nullable = true)
 |-- dow_pickup: string (nullable = true)
 |

### 1.4. Geospatial Analysis

In [29]:
zones.show(20)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [11]:
# Create pickup_borough and pickup_zone first
df = df.join(zones, df["PULocationID"] == zones["LocationID"], "left") \
        .select(df["*"], zones["Borough"].alias("pickup_borough"), zones["Zone"].alias("pickup_zone")) \
        .drop("PULocationID")

In [12]:
# then, create dropoff_borough, dropoff_zone
df = df.join(zones, df["DOLocationID"] == zones["LocationID"], "left") \
        .withColumn("dropoff_borough", col("Borough")) \
        .withColumn("dropoff_zone", col("Zone")) \
        .drop("DOLocationID", "service_zone", "LocationID", "Borough", "Zone")

In [32]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- pickup_hour: string (nullable = true)
 |-- dropoff_hour: string (nullable = true)
 |-- dow_pickup: string (nullable = true)
 |-- dow_dropoff: string (nullable = true)
 |-- month_pickup: string (nullable = true)

#### Summary of 

-----
## 2. Peak Hours and Traffic Analysis
_Investigate the peak hours and traffic patterns based on the pickup and drop-off timestamps. Identify the busiest hours of the day, days of the week, or months of the year. Analyze the average trip duration and fare amount during different time periods to understand the impact of traffic congestion on taxi trips._

In [13]:
df.createOrReplaceTempView('peak')
peak = spark.table('peak')
peak.show(20)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+----------+-----------+------------+-------------+--------------+--------------------+---------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|pickup_hour|dropoff_hour|dow_pickup|dow_dropoff|month_pickup|month_dropoff|pickup_borough|         pickup_zone|dropoff_borough|        dropoff_zone|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+---

### 2.1. Extract Time Information 
_Extract relevant time information from the dataset, such as the pickup or drop-off timestamps. You can use PySpark's date/time functions to extract hour, day of the week, or month information._

In [14]:
peak.select('pickup_hour', 'dropoff_hour', 
            'dow_pickup', 'dow_dropoff', 
            'month_pickup', 'month_dropoff') \
    .show(10)

+-----------+------------+----------+-----------+------------+-------------+
|pickup_hour|dropoff_hour|dow_pickup|dow_dropoff|month_pickup|month_dropoff|
+-----------+------------+----------+-----------+------------+-------------+
|         00|          00|    Sunday|     Sunday|     January|      January|
|         00|          01|    Sunday|     Sunday|     January|      January|
|         00|          00|    Sunday|     Sunday|     January|      January|
|         00|          00|    Sunday|     Sunday|     January|      January|
|         00|          00|    Sunday|     Sunday|     January|      January|
|         00|          01|    Sunday|     Sunday|     January|      January|
|         00|          00|    Sunday|     Sunday|     January|      January|
|         00|          00|    Sunday|     Sunday|     January|      January|
|         00|          00|    Sunday|     Sunday|     January|      January|
|         00|          00|    Sunday|     Sunday|     January|      January|

### 2.2. Calculate Trip Volumes
* Calculate the number of trips for each hour of the day or day of the week. This will help you identify the peak hours or busiest days for taxi trips.
* Compare Weekdays and Weekends: Compare the patterns and trends observed during weekdays and weekends. This analysis can help identify any differences in peak hours or traffic congestion between these time periods.

=> Pick-up time to identify the peak period

In [15]:
peak.groupBy('pickup_hour') \
    .agg(F.count('tpep_pickup_datetime').alias('num_of_trips')) \
    .orderBy('pickup_hour', ascending = True) \
    .show(25)

+-----------+------------+
|pickup_hour|num_of_trips|
+-----------+------------+
|         00|     1003289|
|         01|      671272|
|         02|      439776|
|         03|      284071|
|         04|      177975|
|         05|      191106|
|         06|      475863|
|         07|      956067|
|         08|     1339203|
|         09|     1534347|
|         10|     1680487|
|         11|     1824152|
|         12|     1977300|
|         13|     2036244|
|         14|     2180766|
|         15|     2237222|
|         16|     2238822|
|         17|     2432897|
|         18|     2550783|
|         19|     2287498|
|         20|     2042934|
|         21|     2036656|
|         22|     1874685|
|         23|     1471394|
+-----------+------------+



In [16]:
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

peak2 = peak.groupBy('pickup_hour', 'dow_pickup') \
            .agg(F.count("pickup_hour").alias("trips")) \
            .groupBy("pickup_hour") \
            .pivot("dow_pickup", day_order) \
            .sum("trips") \
            .orderBy("pickup_hour")

In [17]:
peak2.show(25)

+-----------+------+-------+---------+--------+------+--------+------+
|pickup_hour|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday|
+-----------+------+-------+---------+--------+------+--------+------+
|         00| 74577|  68478|    90260|  111151|156060|  247116|255647|
|         01| 38220|  31184|    41294|   54181| 89561|  201324|215508|
|         02| 20057|  15891|    20757|   28772| 51063|  148096|155140|
|         03| 13622|   9665|    12815|   17784| 29597|   94432|106156|
|         04| 14174|  11255|    12878|   15796| 21380|   46240| 56252|
|         05| 28066|  28559|    29048|   29857| 29967|   21356| 24253|
|         06| 75821|  82630|    83892|   84645| 73292|   38824| 36759|
|         07|149291| 178236|   182696|  180877|144751|   62868| 57348|
|         08|202612| 245456|   252810|  249099|196221|  103846| 89159|
|         09|217331| 261184|   265883|  263639|221737|  161578|142995|
|         10|227091| 261665|   268956|  269454|240744|  213460|199117|
|     

In [18]:
peak2a = peak.groupBy('pickup_hour', 'month_pickup') \
            .agg(F.count('tpep_pickup_datetime').alias('num_of_trips'))

month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

peak2a.groupBy('pickup_hour') \
        .pivot('month_pickup', month_order) \
        .sum('num_of_trips') \
        .orderBy('pickup_hour') \
        .show(25)

+-----------+-------+--------+------+------+------+------+------+------+---------+-------+--------+--------+
|pickup_hour|January|February| March| April|   May|  June|  July|August|September|October|November|December|
+-----------+-------+--------+------+------+------+------+------+------+---------+-------+--------+--------+
|         00|  80314|   73117| 87129| 89700| 88126| 94159| 83289| 73355|    72262|  90252|   77981|   93605|
|         01|  56051|   48530| 57038| 60187| 57046| 61449| 57994| 49232|    47051|  61328|   53707|   61659|
|         02|  39069|   33342| 32828| 39329| 36643| 41019| 39237| 32076|    30795|  42256|   33032|   40150|
|         03|  25120|   21494| 24638| 24097| 23473| 25370| 24654| 19936|    20024|  27957|   21643|   25665|
|         04|  15670|   13503| 15526| 14368| 14860| 15900| 14941| 12531|    12685|  17465|   14461|   16065|
|         05|  15925|   14441| 16843| 15432| 17235| 17052| 15301| 15147|    14321|  17912|   16774|   14723|
|         06|  4007

### 2.3. Analyze Trip Durations
* Compute average trip durations for different time periods, such as hours or days of the week. This will allow you to observe any variations in trip durations during peak and off-peak hours.
* Analyze the traffic patterns by exploring the relationship between trip durations and traffic congestion. You can investigate if trip durations tend to be longer during specific hours or days of the week, indicating higher traffic volumes.

In [19]:
from pyspark.sql.functions import expr

peak3 = peak.withColumn('trip_duration',
                        (expr('UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)')/ 60)
                        .cast('double')) 

In [20]:
peak3.groupBy('Pickup_hour') \
        .agg(F.round(F.mean('trip_duration'), 2).alias('mean_trip_duration'), 
             F.count('tpep_pickup_datetime').alias('num_of_trips')) \
        .select('Pickup_hour', 'mean_trip_duration', 'num_of_trips') \
        .orderBy('Pickup_hour') \
        .show(25)

+-----------+------------------+------------+
|Pickup_hour|mean_trip_duration|num_of_trips|
+-----------+------------------+------------+
|         00|             15.79|     1003289|
|         01|             14.98|      671272|
|         02|              13.9|      439776|
|         03|             14.17|      284071|
|         04|             15.49|      177975|
|         05|             16.09|      191106|
|         06|             15.87|      475863|
|         07|             15.77|      956067|
|         08|             16.43|     1339203|
|         09|             16.66|     1534347|
|         10|             17.09|     1680487|
|         11|              17.7|     1824152|
|         12|              18.1|     1977300|
|         13|             18.75|     2036244|
|         14|             19.87|     2180766|
|         15|              20.3|     2237222|
|         16|             20.34|     2238822|
|         17|             18.82|     2432897|
|         18|             16.97|  

In [21]:
peak3.groupBy('dow_pickup') \
        .agg(F.count('tpep_pickup_datetime').alias('num_of_trips'),
             F.round(F.mean('trip_duration'), 2).alias('mean_trip_duration')) \
        .orderBy('dow_pickup') \
        .select('dow_pickup', 'num_of_trips', 'mean_trip_duration') \
        .show(10)

+----------+------------+------------------+
|dow_pickup|num_of_trips|mean_trip_duration|
+----------+------------+------------------+
|    Friday|     5331681|             17.71|
|    Monday|     4487654|             17.03|
|  Saturday|     5218614|             16.58|
|    Sunday|     4498857|             16.57|
|  Thursday|     5644082|             18.26|
|   Tuesday|     5228392|             17.52|
| Wednesday|     5535529|              17.8|
+----------+------------+------------------+



In [22]:
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

peak3.groupBy('pickup_hour') \
        .pivot('dow_pickup', day_order) \
        .mean('trip_duration') \
        .orderBy('pickup_hour') \
        .select('pickup_hour', *[round(col, 2).alias(col) for col in day_order]) \
        .show(25)

+-----------+------+-------+---------+--------+------+--------+------+
|pickup_hour|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday|
+-----------+------+-------+---------+--------+------+--------+------+
|         00| 16.71|   16.6|    15.47|   15.21| 15.47|   15.96| 15.71|
|         01| 16.23|   16.0|    14.58|   14.53| 14.09|   14.64|  15.5|
|         02| 14.52|  15.34|    13.75|   13.38|  13.4|   13.85| 13.99|
|         03| 15.86|  15.46|     13.9|   13.75| 13.66|   13.95| 14.27|
|         04| 16.93|  17.38|     16.7|   16.93| 16.01|   14.57| 14.64|
|         05| 16.34|  15.87|     15.4|   15.95| 16.14|   16.21| 16.86|
|         06| 16.59|  15.65|    15.31|   15.91| 16.45|    15.1| 15.68|
|         07| 17.03|  15.62|    15.55|   15.71| 16.18|   14.15| 14.51|
|         08| 17.13|   17.2|    16.82|   17.03| 16.49|   13.45| 13.26|
|         09| 17.04|   17.9|    17.56|   17.75| 17.29|   13.36| 12.92|
|         10| 17.41|  18.55|    18.42|    18.5| 17.69|   14.03| 13.68|
|     

In [23]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

peak3.groupBy('pickup_hour') \
        .pivot('month_pickup', month_order) \
        .mean('trip_duration') \
        .orderBy('pickup_hour') \
        .select('pickup_hour', *[round(col, 2).alias(col) for col in month_order]) \
        .show(25)       

+-----------+-------+--------+-----+-----+-----+-----+-----+------+---------+-------+--------+--------+
|pickup_hour|January|February|March|April|  May| June| July|August|September|October|November|December|
+-----------+-------+--------+-----+-----+-----+-----+-----+------+---------+-------+--------+--------+
|         00|  15.56|   14.81|15.63|15.91|16.03|15.95|16.09| 15.97|    16.09|  16.25|   15.38|   15.69|
|         01|  14.85|   14.78|15.73|14.62| 14.9|15.38|15.19| 14.45|    14.88|  14.51|    16.8|   13.83|
|         02|  14.59|   13.74|15.01|13.59|13.83|13.81|14.25| 13.82|    14.02|  13.94|   13.19|   13.06|
|         03|  14.47|   14.19|14.84|14.17|14.52|14.14|14.98| 13.92|     14.4|  13.93|   13.56|   12.93|
|         04|  15.61|   14.91|15.06|14.96|14.84| 15.6|15.64| 15.45|    15.82|  16.69|   15.34|   15.74|
|         05|  15.23|   15.66|15.81|16.09|15.24|15.35|15.89| 15.66|    16.75|  17.19|   17.32|   16.81|
|         06|  14.92|   15.65|16.29|15.54|15.71|15.22| 15.0| 15.

### 2.4. Explore Fare Amounts
Analyze the fare amounts during different time periods to understand how fares vary throughout the day or week. You can calculate average fares or observe fare distribution patterns.

In [24]:
peak.groupBy('pickup_hour') \
    .agg(
        F.count('tpep_pickup_datetime').alias('num_of_trips'), 
        F.round(F.mean('fare_amount'), 2).alias('mean_fare')) \
    .orderBy('pickup_hour') \
    .select('pickup_hour', 'num_of_trips', 'mean_fare') \
    .show(25)

+-----------+------------+---------+
|pickup_hour|num_of_trips|mean_fare|
+-----------+------------+---------+
|         00|     1003289|    19.81|
|         01|      671272|    17.87|
|         02|      439776|    16.66|
|         03|      284071|    17.34|
|         04|      177975|    22.11|
|         05|      191106|    25.99|
|         06|      475863|    22.18|
|         07|      956067|    18.89|
|         08|     1339203|    18.15|
|         09|     1534347|    18.31|
|         10|     1680487|    18.63|
|         11|     1824152|    18.75|
|         12|     1977300|    19.06|
|         13|     2036244|    19.91|
|         14|     2180766|    20.49|
|         15|     2237222|    20.54|
|         16|     2238822|    20.53|
|         17|     2432897|    19.25|
|         18|     2550783|    17.95|
|         19|     2287498|    18.11|
|         20|     2042934|    18.68|
|         21|     2036656|    18.87|
|         22|     1874685|    19.65|
|         23|     1471394|    20.72|
+

In [25]:
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

peak.groupBy('pickup_hour') \
    .pivot('dow_pickup', day_order) \
    .mean('fare_amount') \
    .orderBy('pickup_hour') \
    .select('pickup_hour', *[round(col, 2).alias(col) for col in day_order]) \
    .show(25)

+-----------+------+-------+---------+--------+------+--------+------+
|pickup_hour|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday|
+-----------+------+-------+---------+--------+------+--------+------+
|         00| 26.55|  25.32|     21.8|   20.32| 19.23|   17.86| 17.66|
|         01| 25.19|  24.82|    20.68|   19.41| 17.94|   16.13| 16.25|
|         02| 22.18|  23.03|    19.51|   17.77| 16.75|   15.47| 15.81|
|         03| 23.23|  24.45|    20.68|   19.26| 18.07|   15.88|  16.3|
|         04| 27.98|  28.47|    27.47|   25.81| 24.48|   18.63| 19.05|
|         05| 26.81|  24.91|    24.76|   25.14| 26.48|   26.53| 27.79|
|         06| 22.59|  20.49|    19.98|   20.94| 22.48|   25.97| 28.38|
|         07|  19.5|  17.71|    17.59|   17.84| 18.92|   22.32|  24.6|
|         08|  18.6|  17.84|    17.55|   17.61|  18.1|   18.71| 20.67|
|         09| 18.82|  18.49|    18.17|   18.27| 18.71|   17.11| 18.29|
|         10| 19.21|  19.62|    18.79|   18.88| 18.98|   16.76| 17.69|
|     

In [26]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

peak.groupBy('pickup_hour') \
    .pivot('month_pickup', month_order) \
    .mean('fare_amount') \
    .orderBy('pickup_hour') \
    .select('pickup_hour', *[round(col, 2).alias(col) for col in month_order]) \
    .show(25)

+-----------+-------+--------+-----+-----+-----+-----+-----+------+---------+-------+--------+--------+
|pickup_hour|January|February|March|April|  May| June| July|August|September|October|November|December|
+-----------+-------+--------+-----+-----+-----+-----+-----+------+---------+-------+--------+--------+
|         00|  19.57|   18.98|19.68|19.67|20.02|20.52|21.23| 20.42|    20.59|  19.51|    18.8|   18.74|
|         01|  17.61|   16.74|18.24|18.11|18.16|19.07|19.51|  18.2|    18.62|  17.02|   16.41|    16.7|
|         02|  16.43|   16.01|16.81|16.58|16.64|17.67|18.26| 17.15|    17.14|  16.02|   15.61|   15.56|
|         03|  17.15|   16.93|17.43|17.28| 17.3|17.95|18.56| 17.78|    17.51|  16.94|    16.6|   16.65|
|         04|  21.04|   21.57|21.95|22.26|22.37|22.89|22.95| 22.68|    22.45|   21.8|   22.09|   21.47|
|         05|  24.95|   26.16|25.58|26.45|25.75| 25.9|26.58| 25.81|    25.83|  27.17|   25.81|   25.88|
|         06|  21.27|    21.7|22.23| 22.9|22.01|22.22|21.98| 22.

### 2.5. Pickup/Drop-off Locations
* Create heatmaps to visualize the density of pickups or drop-offs across different hours or days. This will help you identify areas with high taxi demand during peak hours.
* Identify Popular Pickup/Drop-off Locations: Determine the most common pickup and drop-off locations during peak hours. This analysis will provide insights into areas with high transportation needs during busy periods.

(điểm trả khách ở đâu => khách thường đi tới đâu => phân khúc khách hàng là ai!!!) 

### 2.6. time periods.
Visualize Results: Use line charts, bar plots, heatmaps, or other suitable visualizations to present your findings and highlight the peak hours and traffic patterns in the NYC trip data.

-----
## 3. Geospatial Analysis
Utilize the pickup and drop-off location data to perform geospatial analysis. Visualize the distribution of taxi trips across different neighborhoods, boroughs, or specific areas of interest. Identify hotspots, popular destinations, and areas with high demand for taxi services.

=> Đi trong thành phố nhiều hơn hay ngoại ô nhiều hơn (borough)

In [29]:
df.createOrReplaceTempView('geo')
geo = spark.table('geo')

In [30]:
geo.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- pickup_hour: string (nullable = true)
 |-- dropoff_hour: string (nullable = true)
 |-- dow_pickup: string (nullable = true)
 |-- dow_dropoff: string (nullable = true)
 |-- month_pickup: string (nullable = true)

### 3.1. The Common Drop-off Borough and Zone

In [31]:
geo1 = geo.groupBy("pickup_borough", "dropoff_borough") \
            .agg(F.count("dropoff_borough").alias("trips_arrived")) \
            .orderBy("pickup_borough") \
            .select("pickup_borough", "dropoff_borough", "trips_arrived")

In [32]:
geo1 = geo.select("pickup_borough","pickup_zone", "dropoff_borough", "dropoff_zone")

-----
##### Create function to input the borough name and then return the value of trips_arrived

In [33]:
# input the dropoff_borough
def within_borough(dropoff_borough):
    filtered_table = geo1.filter(F.col("pickup_borough") == F.col("dropoff_borough")) \
        .filter(F.col("dropoff_borough") == dropoff_borough) \
        .groupBy("pickup_borough", "dropoff_borough", "dropoff_zone") \
        .agg(F.count("dropoff_zone").alias("trips_arrived")) \
        .orderBy(F.asc("pickup_borough"), F.desc("trips_arrived")) \
        .select("pickup_borough", "dropoff_borough", "dropoff_zone", "trips_arrived")
    return filtered_table

# input the pickup_borough
def outside_borough(pickup_borough):
    filtered_table = geo1.filter(F.col("dropoff_borough") != F.col("pickup_borough")) \
                        .filter(F.col("pickup_borough") == pickup_borough) \
                        .groupBy("pickup_borough", "dropoff_borough", "dropoff_zone") \
                        .agg(F.count("dropoff_zone").alias("trips_arrived")) \
                        .orderBy(F.asc("pickup_borough"), F.desc("trips_arrived")) \
                        .select("pickup_borough", "dropoff_borough", "dropoff_zone", "trips_arrived")
    return filtered_table

-----
##### Trips within borough (city)

In [35]:
geo1.filter(F.col("pickup_borough") == F.col("dropoff_borough")) \
    .groupBy("pickup_borough", "dropoff_borough") \
    .agg(F.count("dropoff_borough").alias("trips_arrived")) \
    .orderBy("pickup_borough") \
    .select("pickup_borough", "dropoff_borough", "trips_arrived") \
    .show()

+--------------+---------------+-------------+
|pickup_borough|dropoff_borough|trips_arrived|
+--------------+---------------+-------------+
|         Bronx|          Bronx|        23799|
|      Brooklyn|       Brooklyn|        96420|
|           EWR|            EWR|           85|
|     Manhattan|      Manhattan|     29916253|
|           N/A|            N/A|         1184|
|        Queens|         Queens|       711060|
| Staten Island|  Staten Island|          504|
|       Unknown|        Unknown|       248275|
+--------------+---------------+-------------+



##### The number of trips traveling to zone within borough

In [36]:
geo1.filter(F.col("pickup_borough") == F.col("dropoff_borough")) \
    .groupBy("pickup_borough", "dropoff_borough", "dropoff_zone") \
    .agg(F.count("dropoff_zone").alias("trips_arrived")) \
    .orderBy(F.asc("pickup_borough"), F.desc("trips_arrived")) \
    .select("pickup_borough", "dropoff_borough", "dropoff_zone", "trips_arrived").show()

+--------------+---------------+--------------------+-------------+
|pickup_borough|dropoff_borough|        dropoff_zone|trips_arrived|
+--------------+---------------+--------------------+-------------+
|         Bronx|          Bronx|Van Nest/Morris Park|         1958|
|         Bronx|          Bronx|Mott Haven/Port M...|         1271|
|         Bronx|          Bronx|          Co-Op City|         1067|
|         Bronx|          Bronx|             Norwood|         1038|
|         Bronx|          Bronx|       Melrose South|         1005|
|         Bronx|          Bronx|East Concourse/Co...|          995|
|         Bronx|          Bronx|Soundview/Castle ...|          969|
|         Bronx|          Bronx|      West Concourse|          947|
|         Bronx|          Bronx|Williamsbridge/Ol...|          852|
|         Bronx|          Bronx|          Mount Hope|          835|
|         Bronx|          Bronx|      Pelham Parkway|          742|
|         Bronx|          Bronx|        East Tre

In [37]:
dropoff_borough = geo1.select("dropoff_borough").distinct().collect()

for row in dropoff_borough:
    print(row[0])

Queens
EWR
Unknown
Brooklyn
Staten Island
N/A
Manhattan
Bronx


In [38]:
# automatically return the result
for row in geo1.select("dropoff_borough").distinct().collect():
    dropoff_borough = row[0]
    within_borough(dropoff_borough).show(5)

+--------------+---------------+-----------------+-------------+
|pickup_borough|dropoff_borough|     dropoff_zone|trips_arrived|
+--------------+---------------+-----------------+-------------+
|        Queens|         Queens|      JFK Airport|        69262|
|        Queens|         Queens|LaGuardia Airport|        41545|
|        Queens|         Queens|          Astoria|        35665|
|        Queens|         Queens|     Baisley Park|        31768|
|        Queens|         Queens| South Ozone Park|        29160|
+--------------+---------------+-----------------+-------------+
only showing top 5 rows

+--------------+---------------+--------------+-------------+
|pickup_borough|dropoff_borough|  dropoff_zone|trips_arrived|
+--------------+---------------+--------------+-------------+
|           EWR|            EWR|Newark Airport|           85|
+--------------+---------------+--------------+-------------+

+--------------+---------------+------------+-------------+
|pickup_borough|dro

##### Trips outside borough (city)

In [39]:
geo1.filter(F.col("pickup_borough") != F.col("dropoff_borough")) \
    .groupBy("pickup_borough", "dropoff_borough") \
    .agg(F.count("dropoff_borough").alias("trips_arrived")) \
    .orderBy("pickup_borough") \
    .select("pickup_borough", "dropoff_borough", "trips_arrived") \
    .show(70)

+--------------+---------------+-------------+
|pickup_borough|dropoff_borough|trips_arrived|
+--------------+---------------+-------------+
|         Bronx|         Queens|         3601|
|         Bronx|       Brooklyn|         3728|
|         Bronx|  Staten Island|          175|
|         Bronx|            N/A|          286|
|         Bronx|      Manhattan|        16048|
|         Bronx|        Unknown|           35|
|      Brooklyn|      Manhattan|        63320|
|      Brooklyn|          Bronx|         3466|
|      Brooklyn|            N/A|          332|
|      Brooklyn|         Queens|        17023|
|      Brooklyn|        Unknown|          316|
|      Brooklyn|  Staten Island|          571|
|      Brooklyn|            EWR|            5|
|           EWR|            N/A|           55|
|           EWR|       Brooklyn|            7|
|           EWR|        Unknown|           18|
|           EWR|         Queens|           11|
|           EWR|      Manhattan|           23|
|           E

##### The number of trips traveling to zone outside the borough

In [40]:
geo1.filter(F.col("pickup_borough") != F.col("dropoff_borough")) \
    .groupBy("pickup_borough", "dropoff_borough", "dropoff_zone") \
    .agg(F.count("dropoff_zone").alias("trips_arrived")) \
    .orderBy(F.asc("pickup_borough"), F.desc("trips_arrived")) \
    .select("pickup_borough", "dropoff_borough", "dropoff_zone", "trips_arrived") \
    .show(10)

+--------------+---------------+--------------------+-------------+
|pickup_borough|dropoff_borough|        dropoff_zone|trips_arrived|
+--------------+---------------+--------------------+-------------+
|         Bronx|      Manhattan|Central Harlem North|         1377|
|         Bronx|      Manhattan|   East Harlem South|         1159|
|         Bronx|      Manhattan|   East Harlem North|          993|
|         Bronx|      Manhattan|      Central Harlem|          790|
|         Bronx|      Manhattan|Washington Height...|          744|
|         Bronx|      Manhattan|     Lenox Hill East|          660|
|         Bronx|      Manhattan|            Kips Bay|          604|
|         Bronx|      Manhattan|        East Chelsea|          465|
|         Bronx|      Manhattan|    Hamilton Heights|          442|
|         Bronx|      Manhattan| Morningside Heights|          363|
+--------------+---------------+--------------------+-------------+
only showing top 10 rows



In [41]:
geo1a = geo.groupBy("pickup_borough", "dropoff_borough") \
            .agg(
                F.count("dropoff_borough").alias("trips_arrived"),
                F.round(F.mean("fare_amount"), 2).alias("mean_fare")) \
            .select("pickup_borough", "dropoff_borough", "trips_arrived", "mean_fare")

In [43]:
geo1a.filter(F.col("dropoff_borough") == F.col("pickup_borough")).show(10)

+--------------+---------------+-------------+---------+
|pickup_borough|dropoff_borough|trips_arrived|mean_fare|
+--------------+---------------+-------------+---------+
|           EWR|            EWR|           85|    51.03|
|     Manhattan|      Manhattan|     29916253|     13.8|
|      Brooklyn|       Brooklyn|        96420|    17.82|
|           N/A|            N/A|         1184|    41.68|
|        Queens|         Queens|       711060|     32.4|
|         Bronx|          Bronx|        23799|    21.13|
|       Unknown|        Unknown|       248275|    19.75|
| Staten Island|  Staten Island|          504|     9.52|
+--------------+---------------+-------------+---------+



In [44]:
geo1a.filter(F.col("dropoff_borough") != F.col("pickup_borough")) \
    .orderBy("dropoff_borough") \
    .show(70) 

+--------------+---------------+-------------+---------+
|pickup_borough|dropoff_borough|trips_arrived|mean_fare|
+--------------+---------------+-------------+---------+
|      Brooklyn|          Bronx|         3466|    56.88|
|        Queens|          Bronx|        75461|    62.91|
|       Unknown|          Bronx|          730|    47.16|
|           N/A|          Bronx|           99|    49.77|
|     Manhattan|          Bronx|       100768|    39.07|
| Staten Island|          Bronx|          217|    67.36|
|           EWR|          Bronx|            1|    135.3|
|           EWR|       Brooklyn|            7|     85.6|
| Staten Island|       Brooklyn|          334|    52.47|
|         Bronx|       Brooklyn|         3728|    56.15|
|     Manhattan|       Brooklyn|       733836|    32.34|
|       Unknown|       Brooklyn|         4414|    45.81|
|           N/A|       Brooklyn|          120|    55.53|
|        Queens|       Brooklyn|       537469|    56.75|
|           N/A|            EWR

### 3.2. The Peak and Off Season of Locations

In [45]:
geo2a = geo.groupBy("dropoff_borough", "dow_dropoff") \
            .agg(F.count("tpep_dropoff_datetime").alias("num_of_trips"))

day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

geo2a.groupBy("dropoff_borough") \
    .pivot("dow_dropoff", day_order) \
    .sum("num_of_trips") \
    .orderBy("dropoff_borough") \
    .show()

+---------------+-------+-------+---------+--------+-------+--------+-------+
|dropoff_borough| Monday|Tuesday|Wednesday|Thursday| Friday|Saturday| Sunday|
+---------------+-------+-------+---------+--------+-------+--------+-------+
|          Bronx|  28350|  29636|    30599|   29943|  29836|   27835|  28342|
|       Brooklyn| 175302| 184627|   196266|  203775| 196315|  203754| 216289|
|            EWR|    146|    101|      141|     141|    170|     160|    173|
|      Manhattan|3972006|4705876|  4985491| 5071312|4778594| 4700173|3913796|
|            N/A|  14111|  12633|    12067|   12616|  12445|   10427|  14359|
|         Queens| 255208| 247151|   260488|  274361| 266083|  225675| 281870|
|  Staten Island|   1224|   1358|     1415|    1454|   1372|    1268|   1410|
|        Unknown|  41307|  47010|    49062|   50480|  46866|   49322|  42618|
+---------------+-------+-------+---------+--------+-------+--------+-------+



In [47]:
geo2b = geo.groupBy("dropoff_borough", "month_dropoff") \
            .agg(F.count("dropoff_borough").alias("trips"))

month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

geo2b.groupBy("dropoff_borough") \
    .pivot("month_dropoff", month_order) \
    .sum("trips") \
    .orderBy("dropoff_borough") \
    .show()

+---------------+-------+--------+-------+-------+-------+-------+-------+-------+---------+-------+--------+--------+
|dropoff_borough|January|February|  March|  April|    May|   June|   July| August|September|October|November|December|
+---------------+-------+--------+-------+-------+-------+-------+-------+-------+---------+-------+--------+--------+
|          Bronx|  16179|   14500|  18161|  18013|  18997|  17474|  18025|  19303|    15603|  16794|   15847|   15645|
|       Brooklyn| 111304|  102624| 120463| 119729| 128820| 122647| 107542| 109043|   103676| 123863|  113286|  113331|
|            EWR|    156|     143|    209|    183|     59|     50|     58|     40|       36|     36|      28|      34|
|      Manhattan|2622890| 2489294|2892109|2780703|2957022|2771565|2424408|2337569|  2332122|2923747| 2800193| 2795626|
|            N/A|   6639|    5985|   7292|   7617|   7063|   8093|   8587|   8653|     6916|   7915|    7057|    6841|
|         Queens| 142184|  129131| 159359| 15604

-----
## 4. Fare and Pricing Analysis
Analyze fare amounts and pricing structures. Investigate factors that influence fare variations, such as trip distance, time of day, and rate codes. Explore the impact of additional charges, tolls, and surcharges on the total fare amount. Compare fare amounts between different vendors or payment methods.

In [49]:
df.select("fare_amount").summary().show()

+-------+------------------+
|summary|       fare_amount|
+-------+------------------+
|  count|          35944809|
|   mean|19.261310007517885|
| stddev| 77.41689208909388|
|    min|              0.02|
|    25%|               9.3|
|    50%|              13.5|
|    75%|              21.9|
|    max|         386983.63|
+-------+------------------+



In [50]:
df.createOrReplaceTempView('fare')
fare = spark.table('fare')

In [52]:
fare.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- pickup_hour: string (nullable = true)
 |-- dropoff_hour: string (nullable = true)
 |-- dow_pickup: string (nullable = true)
 |-- dow_dropoff: string (nullable = true)
 |-- month_pickup: string (nullable = true)

### 4.1. The Fare based on Trip Distances

In [54]:
fare = fare.withColumn('tpep_pickup_datetime_utc', F.from_utc_timestamp(fare['tpep_pickup_datetime'], 'UTC')) \
            .withColumn('tpep_dropoff_datetime_utc', F.from_utc_timestamp(fare['tpep_dropoff_datetime'], 'UTC')) \
            .withColumn('time(min)',
                        ((F.col('tpep_dropoff_datetime_utc').cast('long') 
                              - F.col('tpep_pickup_datetime_utc').cast('long')) / 60).cast('double'))

fare = fare.drop('tpep_pickup_datetime_utc', 'tpep_dropoff_datetime_utc')

In [55]:
fare = fare.withColumn('speed', (F.col('trip_distance')/ F.col('time(min)')))

In [56]:
fare.select(round("speed", 2).alias("speed"), 
             round("trip_distance").alias("trip_distance"), 
             round("time(min)").alias("time(min)")) \
        .show()

+-----+-------------+---------+
|speed|trip_distance|time(min)|
+-----+-------------+---------+
| 0.12|          1.0|      8.0|
| 0.17|          1.0|      6.0|
|  0.2|          3.0|     13.0|
|  0.2|          2.0|     10.0|
| 0.13|          1.0|     11.0|
| 0.15|          2.0|     12.0|
| 0.16|          2.0|     10.0|
| 0.51|         12.0|     23.0|
|  0.2|          3.0|     15.0|
| 0.28|          3.0|     11.0|
| 0.21|          2.0|      9.0|
| 0.22|          7.0|     34.0|
| 0.11|          3.0|     30.0|
| 0.57|         11.0|     20.0|
| 0.19|          3.0|     15.0|
| 0.17|          2.0|      9.0|
|  0.1|          2.0|     22.0|
|  0.3|          5.0|     15.0|
|  0.2|          1.0|      6.0|
| 0.17|          3.0|     14.0|
+-----+-------------+---------+
only showing top 20 rows



In [59]:
fare.select("trip_distance").summary().show()

+-------+------------------+
|summary|     trip_distance|
+-------+------------------+
|  count|          35944809|
|   mean|3.4252982398653558|
| stddev| 4.389856609919622|
|    min|              0.01|
|    25%|              1.08|
|    50%|               1.8|
|    75%|              3.38|
|    max|              99.9|
+-------+------------------+



In [61]:
from pyspark.sql.functions import col, floor
bin_size = (99.9 - 0.01)/ 10
fare = fare.withColumn("distance_bin", (floor(col("trip_distance") / bin_size) * bin_size))

In [63]:
fare.groupBy("distance_bin") \
    .agg(F.round(F.mean("fare_amount"), 2).alias("fare")) \
    .select("distance_bin", "fare") \
    .orderBy("distance_bin") \
    .show()

+------------------+------+
|      distance_bin|  fare|
+------------------+------+
|               0.0| 15.17|
|             9.989| 60.79|
|            19.978|  78.8|
|29.967000000000002|141.19|
|            39.956|225.77|
| 49.94500000000001| 275.2|
|59.934000000000005|317.56|
|            69.923|371.08|
|            79.912|448.15|
| 89.90100000000001|454.94|
| 99.89000000000001| 364.2|
+------------------+------+



### 4.2. The Fare based on Locations

In [67]:
fare.filter(F.col("pickup_borough") == F.col("dropoff_borough")) \
    .groupBy("pickup_borough", "dropoff_borough") \
    .agg(F.count("dropoff_borough").alias("trips_arrived"),
         F.round(F.mean("fare_amount"), 2).alias("avg_fare")) \
    .orderBy("pickup_borough") \
    .select("pickup_borough", "dropoff_borough", "trips_arrived", "avg_fare") \
    .show()

+--------------+---------------+-------------+--------+
|pickup_borough|dropoff_borough|trips_arrived|avg_fare|
+--------------+---------------+-------------+--------+
|         Bronx|          Bronx|        23799|   21.13|
|      Brooklyn|       Brooklyn|        96420|   17.82|
|           EWR|            EWR|           85|   51.03|
|     Manhattan|      Manhattan|     29916253|    13.8|
|           N/A|            N/A|         1184|   41.68|
|        Queens|         Queens|       711060|    32.4|
| Staten Island|  Staten Island|          504|    9.52|
|       Unknown|        Unknown|       248275|   19.75|
+--------------+---------------+-------------+--------+



In [71]:
fare.filter(F.col("pickup_borough") != F.col("dropoff_borough")) \
    .groupBy("pickup_borough", "dropoff_borough") \
    .agg(F.count("dropoff_borough").alias("trips_arrived"),
         F.round(F.mean("fare_amount"), 2).alias("avg_fare")) \
    .orderBy("pickup_borough") \
    .select("pickup_borough", "dropoff_borough", "trips_arrived", "avg_fare") \
    .show(70)

+--------------+---------------+-------------+--------+
|pickup_borough|dropoff_borough|trips_arrived|avg_fare|
+--------------+---------------+-------------+--------+
|         Bronx|         Queens|         3601|   44.88|
|         Bronx|       Brooklyn|         3728|   56.15|
|         Bronx|  Staten Island|          175|   68.14|
|         Bronx|            N/A|          286|   52.58|
|         Bronx|      Manhattan|        16048|    32.9|
|         Bronx|        Unknown|           35|    27.2|
|      Brooklyn|      Manhattan|        63320|   31.47|
|      Brooklyn|          Bronx|         3466|   56.88|
|      Brooklyn|            N/A|          332|   81.47|
|      Brooklyn|         Queens|        17023|   42.86|
|      Brooklyn|        Unknown|          316|   28.61|
|      Brooklyn|  Staten Island|          571|   42.21|
|      Brooklyn|            EWR|            5|    94.7|
|           EWR|            N/A|           55|   51.11|
|           EWR|       Brooklyn|            7|  

### 4.3. The Fare Amount during Peak Season

In [77]:
hour = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', 
        '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
        '21', '22', '23']

fare.filter(F.col("pickup_borough") == F.col("dropoff_borough")) \
    .groupBy("dropoff_borough") \
    .pivot("dropoff_hour", hour) \
    .mean("fare_amount") \
    .orderBy("dropoff_borough") \
    .select("dropoff_borough", *[round(col, 2).alias(col) for col in hour]) \
    .show(10)

+---------------+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|dropoff_borough|   00|   01|   02|   03|    04|   05|   06|   07|   08|   09|   10|   11|   12|   13|   14|   15|   16|   17|   18|   19|   20|   21|   22|   23|
+---------------+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|          Bronx|16.67|14.42|16.29|16.44| 19.69|23.79|23.96|23.75|21.27| 21.0|20.94|21.03|21.03|20.87|20.39|20.89|20.99|22.22|20.76|20.94| 20.0| 19.8|19.51|17.25|
|       Brooklyn|15.14| 15.1| 15.4|16.91| 20.21|21.67| 21.1| 19.9| 19.2|19.67|20.08|19.84|20.13|19.59|19.04|18.43|18.67|17.25|16.76| 16.0|15.54|15.07|14.77|14.93|
|            EWR| NULL| NULL| 42.0| NULL|105.25|48.33|56.74|  6.5|  7.2| 10.7|41.47|  3.7|75.25|60.93|  6.5|42.36|71.79| 80.0|57.67|47.86|  3.0| NULL|  3.0|  3.0|
|      Manhattan|13.95

In [80]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

fare.filter(F.col("pickup_borough") == F.col("dropoff_borough")) \
    .groupBy("dropoff_borough") \
    .pivot("month_dropoff", month_order) \
    .mean("fare_amount") \
    .orderBy("dropoff_borough") \
    .select("dropoff_borough", *[round(col, 2).alias(col) for col in month_order]) \
    .show(10)

+---------------+-------+--------+-----+-----+-----+-----+-----+------+---------+-------+--------+--------+
|dropoff_borough|January|February|March|April|  May| June| July|August|September|October|November|December|
+---------------+-------+--------+-----+-----+-----+-----+-----+------+---------+-------+--------+--------+
|          Bronx|  20.88|   20.52|20.92|21.11|21.35|20.79|21.01| 21.02|    20.92|   21.2|   22.01|   21.44|
|       Brooklyn|  18.06|   17.32|17.59|17.49|17.56|17.31|17.48| 18.41|    18.09|  17.94|   17.97|   18.45|
|            EWR|  85.09|   41.22| 65.7| 46.2| 34.9| 53.0|35.43| 59.67|    27.63|  62.16|   43.99|   64.65|
|      Manhattan|  13.03|    13.2|13.39| 13.6|13.99|13.97| 13.6| 13.52|    14.45|  14.28|   14.18|    14.3|
|            N/A|  36.83|    48.6|38.73|44.06|39.54|42.27|48.09| 40.23|    44.31|  46.25|   34.46|   32.55|
|         Queens|  30.38|   31.27|31.83|32.57| 32.8|33.67|33.44| 32.75|    33.09|  32.56|   31.92|   31.96|
|  Staten Island|  15.81|   

In [79]:
hour = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', 
        '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
        '21', '22', '23']

fare.filter(F.col("pickup_borough") != F.col("dropoff_borough")) \
    .groupBy("dropoff_borough") \
    .pivot("dropoff_hour", hour) \
    .mean("fare_amount") \
    .orderBy("dropoff_borough") \
    .select("dropoff_borough", *[round(col, 2).alias(col) for col in hour]) \
    .show(70)

+---------------+------+------+------+-----+------+-----+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+
|dropoff_borough|    00|    01|    02|   03|    04|   05|    06|    07|    08|    09|    10|    11|   12|    13|    14|    15|    16|    17|    18|    19|    20|    21|    22|    23|
+---------------+------+------+------+-----+------+-----+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+
|          Bronx|  49.9| 49.55| 47.99|46.34|  46.0|47.48| 49.42|  48.1| 47.81|  47.5| 48.05|  50.1|46.41| 46.15| 48.54| 51.29| 53.28| 54.37| 51.48|  51.4| 49.59| 49.12| 48.16| 48.64|
|       Brooklyn| 40.57| 36.88| 32.38|30.35| 30.97|41.05| 49.52| 51.83| 50.91| 47.59| 44.87| 44.38|44.02| 44.71| 45.49| 47.35| 48.86| 48.38| 45.43| 43.15| 42.64| 41.21| 40.02| 40.93|
|            EWR|134.25|119.17|105.47|93.48|102.64|98.77|105.05|108.78|112.11|112.94|

In [81]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

fare.filter(F.col("pickup_borough") != F.col("dropoff_borough")) \
    .groupBy("dropoff_borough") \
    .pivot("month_dropoff", month_order) \
    .mean("fare_amount") \
    .orderBy("dropoff_borough") \
    .select("dropoff_borough", *[round(col, 2).alias(col) for col in month_order]) \
    .show(70)

+---------------+-------+--------+------+------+-----+-----+-----+------+---------+-------+--------+--------+
|dropoff_borough|January|February| March| April|  May| June| July|August|September|October|November|December|
+---------------+-------+--------+------+------+-----+-----+-----+------+---------+-------+--------+--------+
|          Bronx|  47.97|   47.39| 48.35| 48.65|49.45|50.61|50.41| 50.73|    50.34|  50.58|   50.04|   48.42|
|       Brooklyn|  41.58|   41.49| 42.04| 42.51|42.54|43.05|43.61| 43.64|    44.02|  43.36|   42.74|   42.01|
|            EWR| 113.97|  107.84|117.77|120.52|96.26| 99.3|97.02| 87.37|    86.76|  88.81|  104.15|    92.2|
|      Manhattan|  52.89|    52.8| 54.98| 55.76|55.72|55.93|55.89| 55.21|    56.72|  55.65|   55.19|   55.78|
|            N/A|  87.69|   92.14| 94.89| 94.99|94.15| 97.2|95.28| 91.61|    95.74|   92.6|   93.76|   92.23|
|         Queens|  45.29|    45.4| 46.77| 47.76|48.88|48.25|47.16|  48.2|    52.32|  49.53|    48.8|    48.9|
|  Staten 

In [ ]:
fare 

In [ ]:
fare.filter